# Continuous Waveform Output

A simple notebook to generate a continuous output from both RFSoC DACs.

User may specify amplitude, frequency and phase of output.

In [ ]:
# Import libraries
%load_ext autoreload
%autoreload 2

%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [ ]:
# Board libraries
from qick import *
import Pyro4

# Pyro communicates with board
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "172.24.106.21" # <-- SET IP ADDRESS
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)


QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Wed Sep  6 18:49:29 2023

	Global clocks (MHz): tProcessor 409.600, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by output 7, pin 14, feedback to tProc input 0
		ADC tile 0, blk 0 is ADC_D
	1:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by o

## Configure Output

In [ ]:
gain = 30000 # [arb.]
freq = 1000 # [MHz.]
phase = 0 # [deg.]

# Program DAC registers
prog = QickProgram(soccfg)
for ch in range(2): 
    prog.declare_gen(ch=ch, nqz=1) # Initialise DACs
    prog.set_pulse_registers(ch = ch,
                             gain = gain,
                             freq = prog.freq2reg(freq, gen_ch=ch),
                             phase = prog.deg2reg(phase, gen_ch=ch),
                             style = "const",
                             length = prog.us2cycles(100, gen_ch=ch),
                             mode = "periodic", # Constant output
                            )
    prog.pulse(ch=ch, t=0) # Trigger DAC

# Start DACs
prog.config_all(soc) # Configure for internal start
soc.tproc.start() # Internal start
# print(prog)

## Reset DACs to stop output or load a new waveform

In [ ]:
soc.reset_gens()